In [ ]:
import QhX

In [ ]:
from QhX.data_manager import DataManager
data_manager = DataManager()
fs_df = data_manager.load_fs_df('https://zenodo.org/record/6878414/files/ForcedSourceTable.parquet')
fs_gp = data_manager.group_fs_df()

In [ ]:
td_objects=data_manager.load_object_df("https://zenodo.org/record/6878414/files/ObjectTable.parquet")
#Find quasars IDs
setindexqso=td_objects[(td_objects["class"].eq("Qso"))].index

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
##FIND quasars indices and transform to arrays
setindexnew=data_manager.get_qso(setindexqso)
setindexnew=np.array(setindexnew)
df = pd.DataFrame({'objectId': setindexnew})
df.set_index('objectId', inplace=True)
setidnew=df.index


In [ ]:
df.head()

In [ ]:
from QhX.light_curve import get_lctiktok, get_lc22
light_curves_data = get_lc22(data_manager, '1384142', include_errors=False)

In [ ]:
from QhX.calculation import *
from QhX.detection import *


In [ ]:
# Ensure to import or define other necessary functions like hybrid2d, periods, same_periods, etc.
from QhX.algorithms.wavelets.wwtz import *
process1_results = process1_new(data_manager, '1384142', ntau=80, ngrid=800, provided_minfq=2000, provided_maxfq=10, include_errors=True)

In [ ]:
from QhX.output import classify_periods, classify_period
outt=classify_periods([process1_results])
outt['classification'] =outt.apply(classify_period, axis=1)
print(outt)

In [ ]:
outt['classification'] =outt.apply(classify_period, axis=1)
print(outt)

In [ ]:
import numpy as np

def generate_sinusoid(time, amplitude, period):
    """
    Generate a sinusoidal signal.

    :param time: Array of time points at which to evaluate the sinusoid.
    :param amplitude: Amplitude of the sinusoid.
    :param period: Period of the sinusoid.
    :return: Array of sinusoidal values.
    """
    return amplitude * np.sin(2 * np.pi * time / period)

def inject_sinus(time, light_curve, period,amplitude_percentage):
    """
    Inject a sinusoidal signal into a quasar red noise light curve.

    :param light_curve: Original quasar red noise light curve.
    :param time: Array of time points corresponding to the light curve data.
    :param amplitude_percentage: Percentage of the light curve's amplitude to use for the sinusoid.
    :param period: Period of the sinusoidal signal to inject.
    :return: Light curve with the sinusoidal signal injected.
    """
    max_amplitude = np.max(light_curve) - np.min(light_curve)
    sinusoid_amplitude = max_amplitude * amplitude_percentage / 100
    sinusoid = generate_sinusoid(time, sinusoid_amplitude, period)
    return light_curve + sinusoid,time


In [ ]:

def get_sinus(data_manager, set1, period, amplitude_percentage):
    """
    Processes light curve data and optionally injects a tik-tok signal based on specified parameters.

    Parameters:
    -----------
     - set1: Identifier for the dataset to process.
     - period (float): Period of the  signal.
     - amplitude_percentage (float): Percentage of the light curve's amplitude to use for the sinusoid.

    Returns:
    --------
    tuple: Processed time and magnitude data for multiple filters, their sampling rates, and tik-tok signals if injected.
    """
    if set1 not in fs_gp.groups:
        print(f"Set ID {set1} not found.")
        return None
       # Retrieve the light curve data for the specified set
    # Retrieve and process the light curve data for the specified set
    demo_lc = fs_gp.get_group(set1)
 
    # Process the data for each filter, sort by MJD, drop rows where MJD is 0 or NaN
    d0, d1, d2, d3 = [
        demo_lc[(demo_lc['filter'] == f) & (demo_lc['mjd'] != 0)].sort_values(by=['mjd']).dropna()
        for f in range(1, 5)
    ]
    # Convert the MJD and magnitude data to numpy arrays for each filter
    tt00, yy00 = d0['mjd'].to_numpy(), d0['psMag'].to_numpy()
    tt11, yy11 = d1['mjd'].to_numpy(), d1['psMag'].to_numpy()
    tt22, yy22 = d2['mjd'].to_numpy(), d2['psMag'].to_numpy()
    tt33, yy33 = d3['mjd'].to_numpy(), d3['psMag'].to_numpy()

    # Remove outliers from each dataset
   # tt0, yy0 = outliers(tt00, yy00)
   # tt1, yy1 = outliers(tt11, yy11)
   # tt2, yy2 = outliers(tt22, yy22)
   # tt3, yy3 = outliers(tt33, yy33)

    # Calculate the mean sampling rate for each dataset
    sampling0, sampling1, sampling2, sampling3 = [np.mean(np.diff(tt)) for tt in [tt00, tt11, tt22, tt33]]

    # Inject the tik-tok signal into the light curve data if specified
    yy0, tik0 = inject_sinus(tt00, yy00, period, amplitude_percentage)
    yy1, tik1 = inject_sinus(tt11, yy11, period,amplitude_percentage)
    yy2, tik2 = inject_sinus(tt22, yy22, period, amplitude_percentage)
    yy3, tik3 = inject_sinus(tt33, yy33, period, amplitude_percentage)

    # Return the processed data
    return tt00, yy0, tt11, yy1, tt22, yy2, tt33, yy3, sampling0, sampling1, sampling2, sampling3, tik0, tik1, tik2, tik3


In [ ]:
tt0, yy0, tt1, yy1, tt2, yy2, tt3, yy3, sampling0, sampling1, sampling2, sampling3, tik0, tik1, tik2, tik3=get_sinus(fs_gp, '1384142', 50,0.2)

In [ ]:
sampling0,sampling1,sampling2,sampling3

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(tt0,yy0,'ko')

In [ ]:
from QhX.algorithms.wavelets.wwtz import *

In [ ]:
import pandas as pd
import logging

class DataManager:
    def __init__(self):
        self.fs_df = None
        self.fs_gp = None
        self.object_df = None
        self.td_objects = None

    def load_fs_df(self, path_source: str, format: str = 'parquet') -> pd.DataFrame:
        """
        Load data from a file with support for multiple formats.

        Parameters
        ----------
        path_source : str
            The path to the data file.
        format : str, optional
            The format of the file (default is 'parquet').

        Returns
        -------
        pd.DataFrame or None
            The loaded DataFrame or None in case of an error.
        """
        try:
            if format == 'parquet':
                df = pd.read_parquet(path_source)
            elif format == 'csv':
                df = pd.read_csv(path_source)
            else:
                logging.error(f"Unsupported file format: {format}")
                return None
            logging.info(f"Data loaded successfully from {path_source}.")
            return df
        except Exception as e:
            logging.error(f"Error loading data: {e}")
            return None

    def group_fs_df(self, df: pd.DataFrame, group_by: str) -> pd.core.groupby.DataFrameGroupBy:
        """
        Group data by a specified column.

        Parameters
        ----------
        df : pd.DataFrame
            The DataFrame to group.
        group_by : str
            The column name to group by.

        Returns
        -------
        pd.core.groupby.DataFrameGroupBy or None
            The grouped DataFrame or None if the DataFrame is not available.
        """
        if df is not None:
            grouped = df.groupby(group_by)
            logging.info("Data grouped successfully.")
            return grouped
        else:
            logging.warning("DataFrame is not available for grouping.")
            return None

    def get_qso(self, grouped_data: pd.core.groupby.DataFrameGroupBy, object_ids: list, min_points: int = 100, filters: list = None) -> list:
        """
        Get QSOs with complete light curves for specified filters with at least 'min_points' points.

        Parameters
        ----------
        grouped_data : pd.core.groupby.DataFrameGroupBy
            The grouped DataFrame.
        object_ids : list
            List of object IDs to check.
        min_points : int, optional
            Minimum number of points required in each light curve (default is 100).
        filters : list, optional
            List of filters to check (default is None, which checks all filters).

        Returns
        -------
        list
            List of QSO IDs that meet the criteria.
        """
        valid_qsos = []
        for obj_id in object_ids:
            if obj_id in grouped_data.groups:
                lc = grouped_data.get_group(obj_id)
                if filters is None:
                    filters = lc['filter'].unique()
                if all(len(lc[lc['filter'] == f].dropna()) >= min_points for f in filters):
                    valid_qsos.append(obj_id)
        return valid_qsos

# Initialize logging
logging.basicConfig(level=logging.INFO)


In [ ]:
# Initialize logging
logging.basicConfig(level=logging.INFO)
data_manager = DataManager()


In [ ]:
data_manager.group_fs_df

In [ ]:
from QhX.data_manager import DataManager
data_manager = DataManager()
fs_df = data_manager.load_fs_df('https://zenodo.org/record/6878414/files/ForcedSourceTable.parquet')
fs_gp = data_manager.group_fs_df()

In [ ]:
fs_df = data_manager.load_fs_df('https://zenodo.org/record/6878414/files/ForcedSourceTable.parquet', format='parquet')



In [ ]:
if fs_df is not None:
    fs_gp = data_manager.group_fs_df(fs_df, 'objectId')



In [ ]:
# Load the object data from a Parquet file
object_df = data_manager.load_fs_df("https://zenodo.org/record/6878414/files/ObjectTable.parquet", format='parquet')

# Check if the data is loaded and has the 'class' column
if object_df is not None and 'class' in object_df.columns:
    # Find quasars IDs
    quasar_df = object_df[object_df['class'] == 'Qso']

    # Get the indices or IDs of the quasars
    quasar_ids = quasar_df.index.tolist()
 

In [ ]:
setindexqso =object_df[object_df["class"] == "Qso"].index.tolist()

# If additional filtering is needed (e.g., based on light curve data), you can use get_qso
# For now, assuming setindexqso contains the IDs we need and we don't have to call get_qso
# because get_qso method in the new DataManager needs grouped data and specific criteria

# Convert the list of QSO IDs to a numpy array
setindexnew = np.array(setindexqso)

# Create a DataFrame with the quasar IDs
df = pd.DataFrame({'objectId': setindexnew})

# Set the 'objectId' column as the index of the DataFrame
df.set_index('objectId', inplace=True)

# Now df.index contains the quasar IDs as the DataFrame index
setidnew = df.index

In [ ]:
def get_lc_general(fs_df, object_id, include_errors=True):
    """
    Process and return light curves with an option to include magnitude errors for a given object ID.

    Parameters:
    -----------
    - data_manager (DataManager): The DataManager instance with loaded light curve data.
    - object_id (str): The object ID for which light curves are to be processed.
    - include_errors (bool, optional): Flag to include magnitude errors in the time series. Defaults to True.

    Returns:
    --------
    tuple: Contains the processed time series with or without magnitude errors for each
           filter, along with their respective sampling rates. Returns None if the object ID
           is not found or if any filter data is missing.
    """
    if fs_df is None or object_id not in fs_df['objectId'].unique():
        print(f"Object ID {object_id} not found.")
        return None

    demo_lc = fs_df[fs_df['objectId'] == object_id]
    if demo_lc.empty:
        print(f"No light curve data found for object ID {object_id}.")
        return None

    results = []

    for filter_value in range(1, 5):  # Assuming filters range from 1 to 4
        d = demo_lc[demo_lc['filter'] == filter_value].sort_values(by=['mjd']).dropna()
        if d.empty or ('psMagErr' not in d.columns and include_errors):
            print(f"No data or 'err' column not found for filter {filter_value} for object ID {object_id}.")
            return None

        tt, yy = d['mjd'].to_numpy(), d['psMag'].to_numpy()
        err_mag = d['psMagErr'].to_numpy() if 'psMagErr' in d.columns and include_errors else None

        # Apply any data cleaning or outlier removal here, if necessary
        # For now, we assume a function `outliers_mad` exists for this purpose
        # if include_errors and err_mag is not None:
        #     tt, yy, err_mag = outliers_mad(tt, yy, err_mag)
        # else:
        #     tt, yy = outliers_mad(tt, yy)
        yy,_= inject_sinus(tt, yy, 50, 0.2)
        ts_with_or_without_errors = yy

        if include_errors and err_mag is not None:
            ts_with_or_without_errors = yy + np.random.normal(0, err_mag)

        sampling_rate = np.mean(np.diff(tt)) if len(tt) > 1 else 0

        results.extend([tt, ts_with_or_without_errors, sampling_rate])

    return tuple(results)


In [ ]:
    # Inject the tik-tok signal into the light curve data if specified
    yy0, tik0 = inject_sinus(tt00, yy00, period, amplitude_percentage)
    yy1, tik1 = inject_sinus(tt11, yy11, period,amplitude_percentage)
    yy2, tik2 = inject_sinus(tt22, yy22, period, amplitude_percentage)
    yy3, tik3 = inject_sinus(tt33, yy33, period, amplitude_percentage)



In [ ]:
results=get_lc_general(fs_df, '1384142', include_errors=True)

In [ ]:
 tt0, yy0, sampling0, tt1, yy1,sampling1, tt2, yy2, sampling2, tt3, yy3,  sampling3 =results

sampling0

In [ ]:
sampling0,sampling1,sampling2,sampling3

In [ ]:
def process1_new(DataManager, set1, ntau=None, ngrid=None, provided_minfq=None, provided_maxfq=None, include_errors=True, parallel=False):
    """
    Processes and analyzes light curve data from a single object to detect common periods across different bands.

    The process involves:
    
    - Verifying the existence of the dataset.
    - Retrieving and processing light curve data for different bands.
    - Applying hybrid wavelet techniques to each band's light curve data.
    - Comparing periods detected in different bands to find common periods, if they do not differ more than 10%.
    - Compiling the results, including period values, errors, and significance, into a structured format.

    Parameters
    ----------
    set1 : int
        An identifier representing the dataset to be processed.
    ntau : int, optional
        Number of time delays in the wavelet analysis.
    ngrid : int, optional
        Number of grid points in the wavelet analysis.
    provided_minfq : float, optional
        Period correspondig to the Minimum frequency for analysis, default is calculated from data.
    provided_maxfq : float, optional
        Period corresponding to the Maximum frequency for analysis, default is calculated from data.
    include_errors : bool, optional
        Include magnitude errors in analysis. Defaults to True.
 
    Returns
    -------
    A list of dictionaries representing the results of the analysis performed on light curve data. Each dictionary contains:
    
        - objectid (int): Identifier of the object ID.
        - sampling_i (float): Mean sampling rate in the first band of the pair where a common period is detected.
        - sampling_j (float): Mean sampling rate in the second band in the pair.
        - period (float): Detected common period between the two bands. NaN if no period is detected.
        - upper_error (float): Upper error of the detected period. NaN if no period is detected.
        - lower_error (float): Lower error of the detected period. NaN if no period is detected.
        - significance (float): Measure of the statistical significance of the detected period. NaN if no period is detected.
        - label (str): Label identifying the pair of bands where the period was detected (e.g., '0-1', '1-2').

    Example Usage
    -------------
    # results = process1_new(data_manager, '1384142', ntau=80, ngrid=800, provided_minfq=2000, provided_maxfq=10, include_errors=False)
    # df = pd.DataFrame(results)
    # df.to_csv('light_curve_analysis.csv', index=False)
    """
    
    # Assume get_lc_general returns a tuple for each band: (time, magnitude, sampling_rate, errors)
    light_curves_data = get_lc_general(DataManager, set1, include_errors)
    print('a')
    if light_curves_data is None:
        print(f"No data found for object ID {object_id}.")
        return None

    tt0, yy0, sampling0, tt1, yy1,sampling1, tt2, yy2, sampling2, tt3, yy3,  sampling3 = light_curves_data
    results = []
    for tt, yy in [(tt0, yy0), (tt1, yy1), (tt2, yy2), (tt3, yy3)]:
        wwz_matrix, corr, extent = hybrid2d(tt, yy, ntau=ntau, ngrid=ngrid, minfq=provided_minfq, maxfq=provided_maxfq, parallel=parallel)
        peaks, hh, r_periods, up, low = periods(set1, corr, ngrid=ngrid, plot=False, minfq=provided_minfq, maxfq=provided_maxfq)
        results.append((r_periods, up, low, peaks, hh))

    sampling_rates = [sampling0, sampling1, sampling2, sampling3]
    light_curve_labels = ['0', '1', '2', '3']

    det_periods = []
    for i in range(len(results)):
        for j in range(i + 1, len(results)):
            r_periods_i, up_i, low_i, peaks_i, hh_i = results[i]
            r_periods_j, up_j, low_j, peaks_j, hh_j = results[j]

            r_periods_common, u_common, low_common, sig_common = same_periods(
                r_periods_i, r_periods_j, up_i, low_i, up_j, low_j, peaks_i, hh_i, tt0, yy0, peaks_j, hh_j, tt1, yy1,
                ntau=ntau, ngrid=ngrid, minfq=provided_minfq, maxfq=provided_maxfq
            )

            if len(r_periods_common) == 0:
                det_periods.append({
                    "objectid": set1,
                    "sampling_i": sampling_rates[i],
                    "sampling_j": sampling_rates[j],
                    "period": np.nan,
                    "upper_error": np.nan,
                    "lower_error": np.nan,
                    "significance": np.nan,
                    "label": f"{light_curve_labels[i]}-{light_curve_labels[j]}"
                })
            else:
                for k in range(len(r_periods_common)):
                    det_periods.append({
                        "objectid": set1,
                        "sampling_i": sampling_rates[i],
                        "sampling_j": sampling_rates[j],
                        "period": r_periods_common[k],
                        "upper_error": u_common[k],
                        "lower_error": low_common[k],
                        "significance": sig_common[k],
                        "label": f"{light_curve_labels[i]}-{light_curve_labels[j]}"
                    })

    return det_periods


In [ ]:
from QhX.calculation import *
from QhX.detection import *


In [ ]:
# Ensure to import or define other necessary functions like hybrid2d, periods, same_periods, etc.
from QhX.algorithms.wavelets.wwtz import *

In [ ]:
process1_results = process1_new(fs_df, '1384142', ntau=80, ngrid=800, provided_minfq=2000, provided_maxfq=10, include_errors=True)

In [ ]:
from QhX.output import classify_periods, classify_period
outt=classify_periods([process1_results])
outt['classification'] =outt.apply(classify_period, axis=1)
print(outt)

In [ ]:
pip install fastparquet

In [ ]:
light_curves_data = get_lc22(data_manager, '1384142', include_errors=False)

In [1]:
data_manager.fs_gp.groups

NameError: name 'data_manager' is not defined